In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew, LLM, Process
import os

os.environ["GROQ_API_KEY"] = "YOUR_GROQ_API_KEY"

llm=LLM(model="groq/llama3-8b-8192",max_tokens=500) # Model might have been deprecated on Groq, please update to latest Llama model!



In [3]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

os.environ["SERPER_API_KEY"]="YOUR_SERPER_KEY"
search_tool=SerperDevTool()
scrape_tool=ScrapeWebsiteTool()

D:\Coding-Configs\Python Config\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
D:\Coding-Configs\Python Config\Lib\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
D:\Coding-Configs\Python Config\Lib\site-packages\crewai_tools\tools\selenium_scraping_tool\selenium_scraping_tool.py:26: PydanticDeprecatedSince20: 

In [5]:
health_analyst = Agent(
    role="Medical Dietician specializing in disease specific diets if the patient has no disease then suggest "
    " general toxics to avoid."
    "You will write a search query which is string not a dictionary for tool.",
    goal="Given a patient's diagnosed disease, " 
    "provide a detailed list of foods and ingredients to avoid to manage their condition and improve their health.",
    tools=[search_tool, scrape_tool],
    verbose=True,
    allow_delegation=False,
    llm=llm,
    backstory=(
        "A highly experienced medical dietician from India with over 15 years of expertise in creating personalized dietary plans for patients with various diseases. "
        "They have a deep understanding of the impact of food on health and are skilled at identifying potential dietary triggers and recommending suitable alternatives. "
        "They utilize medical research and dietary guidelines to provide evidence-based recommendations."
    ),
)

In [6]:
nutritionist = Agent(
    role="Registered Dietitian specializing in personalized dietary plans"
    "You will write a search query which is string not a dictionary for tool.",
    goal="Based on a patient's health condition, dietary restrictions, and weight goals, recommend a comprehensive and personalized diet plan that promotes recovery, weight management, and overall well-being.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    backstory=(
        "A highly experienced Registered Dietitian from India with over 10 years of expertise in creating personalized dietary plans for patients with diverse health needs. "
        "They specialize in developing evidence-based nutrition plans that consider individual health conditions, dietary restrictions, and weight management goals. "
        "They are skilled in translating complex nutritional information into practical and actionable dietary recommendations."
    ),
)

In [7]:
diet_plan_communicator = Agent(
    role="Nutritional Content Writer specializing in simplifying diet plans for laypersons"
    "You will write a search query which is string not a dictionary for tool.",
    goal="Translate complex dietary recommendations into clear, concise, and easily understandable diet plans for laypersons, ensuring the information is presented in a structured and positive manner.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    backstory=(
        "A skilled nutritional content writer from India with a passion for making dietary information accessible to everyone. "
        "They have extensive experience in translating complex nutritional concepts into simple and actionable diet plans. "
        "They are known for their ability to present information in a structured and positive manner, promoting healthy eating habits and overall well-being."
    ),
)

In [8]:
health_task = Task(
    description="Given {disease} recommend food items to avoid. If the patient has no disease recommend general toxins to avoid."
    "You will always write a search query which is a string and not a dictionary for tool.",
    expected_output="A detailed list of food items and ingredients to avoid, tailored to the patient's diagnosed disease or, in the absence of a diagnosis, a list of common toxic foods.",
    agent=health_analyst,
)

In [9]:
diet_plan=Task(
    description="Create a diet plan for patient "
    " consider to avoid-list of doctor. Consider {weight} and {age}"
    "You will always write a search query which is a string and not a dictionary for tool.",
    expected_output="A concise diet plan for patient.",
    agent=nutritionist
    )

In [10]:
writeup = Task(
    description="Create a clear and concise writeup of the diet plan provided by the nutritionist, ensuring it is easily understandable for a layperson." 
    "Present the information in a structured and positive tone, highlighting key dietary recommendations and benefits."
    "You will always write a search query which is a string and not a dictionary for tool.",
    expected_output="A structured and easy-to-understand document (or formatted text) that presents the nutritionist's diet plan in a clear and positive manner, including meal suggestions, nutritional information, and key takeaways.",
    agent=diet_plan_communicator
)

In [11]:
diet_crew=Crew(
    agents=[health_analyst,nutritionist,diet_plan_communicator],
    tasks=[health_task,diet_plan,writeup],
    verbose=False,
)

In [12]:
patient_details = {
    'disease':"hypertension",  # Or 'diabetes', 'hypertension', etc.
    'age':26,
    'weight':90
}

In [13]:
print("Inputs Sent to CrewAI:", patient_details)  # Ensure it's correctly structured

Inputs Sent to CrewAI: {'disease': 'hypertension', 'age': 26, 'weight': 90}


In [14]:
result = diet_crew.kickoff(inputs=patient_details)

# Agent: Medical Dietician specializing in disease specific diets if the patient has no disease then suggest  general toxics to avoid.You will write a search query which is string not a dictionary for tool.
## Task: Given hypertension recommend food items to avoid. If the patient has no disease recommend general toxins to avoid.You will always write a search query which is a string and not a dictionary for tool.


# Agent: Medical Dietician specializing in disease specific diets if the patient has no disease then suggest  general toxics to avoid.You will write a search query which is string not a dictionary for tool.
## Thought: Thought: Patient has hypertension, need to recommend food items to avoid.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"foods to avoid for hypertension\"}"
## Tool Output: 
{'searchParameters': {'q': 'foods to avoid for hypertension', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'The best and wor

In [19]:
from IPython.display import Markdown, display
op=str(result)
display(Markdown(op))

**Personalized Diet Plan for Overall Health and Wellness**

We are excited to share this customized diet plan, tailored to your unique needs and health goals. This comprehensive plan is designed to promote overall health, weight management, and recovery, while excluding foods that may be detrimental to your health.

**Food Choices to Avoid**

To support your health, it is essential to avoid the following foods:

* Red meat
* Processed foods
* Foods high in added sugars
* Foods high in saturated and trans fats

**Recommended Food Choices**

To nourish your body, focus on the following foods:

* Whole grains, such as brown rice, quinoa, and whole-grain bread
* Lean protein sources, like grilled chicken, fish, and lentils
* Fresh fruits and vegetables, aiming for a variety of colors
* Healthy fats, such as nuts, seeds, avocado, and olive oil
* Low-fat or fat-free dairy products and 100% fruit juice
* Herbs and spices for added flavor instead of salt and sugar

**Sample Meal Plan**

Here's a sample meal plan to get you started:

**Breakfast**

* Oatmeal with fruits and nuts
* Whole-grain toast with avocado and eggs

**Lunch**

* Grilled chicken or fish with quinoa and steamed vegetables
* Lentil soup with whole-grain bread and a side salad

**Dinner**

* Grilled vegetable skewers with brown rice and a side of beans
* Baked salmon with sweet potato and green beans

**Snacks**

* Fresh fruits and cut vegetables with hummus
* Nuts and seeds such as almonds and pumpkin seeds

**Beverages**

* Drink at least 8-10 glasses of water per day
* Unsweetened tea, black coffee, and low-fat or fat-free milk
* 100% fruit juice in moderation

**Additional Recommendations**

* Engage in physical activity, such as walking or gentle exercises, for at least 30 minutes a day
* Consider incorporating probiotics and omega-3 fatty acid supplements, as advised by your doctor
* Monitor and adjust your diet plan as needed, based on your progress and health condition

By following this personalized diet plan, you can expect to enjoy improved overall